In [57]:
styling = [
    (1,1,1,3000),
    (1,1,2,4000),
    (1,2,3,2000),
    (1,3,3,5000),
    (1,4,4,1000),
    (1,5,5,7000),
    (1,6,6,500),
    (1,8,8,2000),
    (1,8,7,1500),
    (1,7,8,1300),
    (2,1,1,1700),
    (2,1,2,1200)
]

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, first

In [59]:
spark = SparkSession.builder.master("local").appName("learn-sql").getOrCreate()

In [60]:
stylingSchema = [ "id", "nickname", "card_id", "score"]

In [61]:
df = spark.createDataFrame(data=styling, schema=stylingSchema)

In [62]:
df.createOrReplaceTempView("styling")

In [67]:
from pyspark.sql.functions import col
windowSpec = Window.partitionBy("id", "nickname").orderBy(col("score").desc(), "card_id")
nicknameWindowSpec = Window.partitionBy("id", "card_id").orderBy(col("score").desc(), "nickname")

testDF = spark.sql("select * from styling order by score DESC")\
    .withColumn("first_card_id", first("card_id").over(windowSpec))\
    .withColumn("first_nickname", first("nickname").over(nicknameWindowSpec))

testDF.show()
testDF.createOrReplaceTempView("styling")

+---+--------+-------+-----+-------------+--------------+
| id|nickname|card_id|score|first_card_id|first_nickname|
+---+--------+-------+-----+-------------+--------------+
|  1|       1|      1| 3000|            2|             1|
|  1|       1|      2| 4000|            2|             1|
|  1|       3|      3| 5000|            3|             3|
|  1|       2|      3| 2000|            3|             3|
|  1|       4|      4| 1000|            4|             4|
|  1|       5|      5| 7000|            5|             5|
|  1|       6|      6|  500|            6|             6|
|  1|       8|      7| 1500|            8|             8|
|  1|       8|      8| 2000|            8|             8|
|  1|       7|      8| 1300|            8|             8|
|  2|       1|      1| 1700|            1|             1|
|  2|       1|      2| 1200|            1|             1|
+---+--------+-------+-----+-------------+--------------+



In [68]:
query = "select * from styling WHERE first_card_id = card_id AND first_nickname = nickname"
spark.sql(query).show()

+---+--------+-------+-----+-------------+--------------+
| id|nickname|card_id|score|first_card_id|first_nickname|
+---+--------+-------+-----+-------------+--------------+
|  1|       1|      2| 4000|            2|             1|
|  1|       3|      3| 5000|            3|             3|
|  1|       4|      4| 1000|            4|             4|
|  1|       5|      5| 7000|            5|             5|
|  1|       6|      6|  500|            6|             6|
|  1|       8|      8| 2000|            8|             8|
|  2|       1|      1| 1700|            1|             1|
+---+--------+-------+-----+-------------+--------------+



In [77]:
windowSpec = Window.partitionBy("id").orderBy("id")

In [78]:
query = "select * from styling"
spark.sql(query)\
    .withColumn("rank", row_number().over(windowSpec))\
    .show()

+---+--------+-------+----+
| id|nickname|card_id|rank|
+---+--------+-------+----+
|  1|       1|      1|   1|
|  1|       1|      2|   2|
|  1|       2|      3|   3|
|  2|       1|      1|   1|
|  2|       1|      2|   2|
+---+--------+-------+----+



In [79]:
windowSpec = Window.partitionBy("id", "nickname").orderBy("card_id")
query = "select * from styling"

testDF = spark.sql(query)\
    .withColumn("rank", row_number().over(windowSpec))\
    .withColumn("first_card_id", first("card_id").over(windowSpec))

testDF.show()

+---+--------+-------+----+-------------+
| id|nickname|card_id|rank|first_card_id|
+---+--------+-------+----+-------------+
|  1|       1|      1|   1|            1|
|  1|       1|      2|   2|            1|
|  1|       2|      3|   1|            3|
|  2|       1|      1|   1|            1|
|  2|       1|      2|   2|            1|
+---+--------+-------+----+-------------+



In [80]:
testDF.createOrReplaceTempView("styling2")

In [81]:
query = "select * from styling2 where card_id = first_card_id"

spark.sql(query).show()

+---+--------+-------+----+-------------+
| id|nickname|card_id|rank|first_card_id|
+---+--------+-------+----+-------------+
|  1|       1|      1|   1|            1|
|  1|       2|      3|   1|            3|
|  2|       1|      1|   1|            1|
+---+--------+-------+----+-------------+

